# 1. Introducción general  

El método de Monte Carlo es una técnica computacional basada en muestreos aleatorios repetidos para aproximar soluciones numéricas de problemas que pueden ser difíciles o imposibles de resolver analíticamente. Se utiliza, por ejemplo, para estimar integrales, resolver problemas de optimización o estudiar sistemas con incertidumbre. De forma general, el método consiste en:

1. Definir una región o un conjunto de parámetros.  
2. Tomar muestras al azar dentro de ese espacio.  
3. Calcular una función determinista en cada valor muestreado.  
4. Promediar los resultados para aproximar la solución buscada.

## Formulación matemática

Para ilustrar el método, considérese el problema de integración unidimensional:

$$
I = \int_a^b f(x)\,dx.
$$

Si se eligen $N$ puntos $x_i$ aleatoriamente y de manera uniforme en $[a,b]$, la estimación de Monte Carlo para $I$ es:

$$
\hat{I} \approx (b - a) \,\frac{1}{N} \sum_{i=1}^N f(x_i).
$$

A medida que $N$ crece, $\hat{I}$ se aproxima al valor real de la integral gracias a la ley de los grandes números. Este mismo principio se extiende a integrales en múltiples dimensiones muestreando en un dominio de dimensión superior y multiplicando por el volumen correspondiente.

## Ejemplos en Python

### Ejemplo en 1D  

In [1]:
import numpy as np

def monte_carlo_1d(func, a, b, num_samples):
    # Generar muestras en [a, b]
    samples = np.random.uniform(a, b, num_samples)
    # Evaluar la función en cada muestra
    evaluations = func(samples)
    # Calcular el promedio y multiplicar por la longitud [b - a]
    integral = (b - a) * np.mean(evaluations)
    return integral

# Ejemplo: f(x) = x^2 en [0,1]
def f_1d(x):
    return x**2

result_1d = monte_carlo_1d(f_1d, 0, 1, 10000)
print("Resultado Monte Carlo 1D:", result_1d)

Resultado Monte Carlo 1D: 0.337046628956089


### Ejemplo en 2D

In [2]:
import numpy as np

def monte_carlo_2d(func, x_bounds, y_bounds, num_samples):
    # Generar muestras en [x_bounds[0], x_bounds[1]] y [y_bounds[0], y_bounds[1]]
    x_samples = np.random.uniform(x_bounds[0], x_bounds[1], num_samples)
    y_samples = np.random.uniform(y_bounds[0], y_bounds[1], num_samples)
    # Evaluar la función
    evaluations = func(x_samples, y_samples)
    # Calcular el área y aproximar la integral
    area = (x_bounds[1] - x_bounds[0]) * (y_bounds[1] - y_bounds[0])
    integral = area * np.mean(evaluations)
    return integral

# Ejemplo: f(x, y) = x^2 + y^2 en [0,1] x [0,1]
def f_2d(x, y):
    return x**2 + y**2

result_2d = monte_carlo_2d(f_2d, (0, 1), (0, 1), 10000)
print("Resultado Monte Carlo 2D:", result_2d)

Resultado Monte Carlo 2D: 0.6682082710916482


### Ejemplo en N Dimensiones

In [3]:
import numpy as np

def monte_carlo_nd(func, bounds, num_samples):
    dimensions = len(bounds)
    # Generar muestras aleatorias en cada dimensión
    samples = np.zeros((num_samples, dimensions))
    for i in range(dimensions):
        low, high = bounds[i]
        samples[:, i] = np.random.uniform(low, high, num_samples)
    # Evaluar la función en cada muestra
    evaluations = func(samples)
    # Calcular el volumen del hiper-rectángulo
    volume = 1
    for low, high in bounds:
        volume *= (high - low)
    # Aproximar la integral
    integral = volume * np.mean(evaluations)
    return integral

# Ejemplo: f(x, y, z) = x^2 + y^2 + z^2 en [0,1]^3
def f_nd(points):
    return np.sum(points**2, axis=1)

bounds_3d = [(0, 1), (0, 1), (0, 1)]
result_nd = monte_carlo_nd(f_nd, bounds_3d, 10000)
print("Resultado Monte Carlo 3D:", result_nd)

Resultado Monte Carlo 3D: 0.9871917562783714


## Ejercicios

**Ejercicio 1**  

- Implementa la integración de Monte Carlo en 1D para una función sencilla (por ejemplo, f(x) = x² en el intervalo [0,1]).  
- Varía el número de muestras (e.g., 100, 1000, 10000) y compara cómo cambia la estimación de la integral.  
- Mide el tiempo de ejecución para cada variación y observa el compromiso entre exactitud y tiempo de cómputo.

**Ejercicio 2**  

- Emplea un método de Monte Carlo en 2D para estimar el área de un cuarto de círculo de radio 1 dentro del cuadrado [0,1]×[0,1].  
- Muestra puntos uniformemente en esa región y cuenta cuántos caen dentro de x² + y² ≤ 1.  
- Observa cómo la estimación se acerca a π/4 conforme aumenta el número de muestras.  
- Representa los puntos de un experimento para visualizar la dispersión.

**Ejercicio 3**  

- Modifica el ejemplo en 2D para aproximar la integral de f(x, y) = sin(x)*cos(y) en algún dominio, por ejemplo [0,1]×[0,1].  
- Compara el resultado calculado por Monte Carlo con algún método de integración numérica más preciso (por ejemplo, usando librerías de Python).  
- Observa la convergencia de la estimación al aumentar el número de muestras.

**Ejercicio 4**  

- Extiende la integración de Monte Carlo a N dimensiones para la función f(x₁, x₂, …, xₙ) = ∑(xᵢ²) en la hipercaja [0,1]ⁿ.  
- Comienza con n = 3 y aumenta hasta n = 5 o más.  
- Observa cómo varía la precisión con el incremento de la dimensión manteniendo fijo el número de muestras.  
- Explica brevemente el fenómeno conocido como “la maldición de la dimensionalidad” y cómo afecta a los métodos de Monte Carlo.

# 2. Muestreo de Importancia

Los métodos de Monte Carlo permiten aproximar cantidades de interés mediante muestreo repetido de una distribución de probabilidad. El muestreo de importancia (importance sampling) es una técnica que puede mejorar la eficiencia concentrando las muestras en las regiones más “relevantes” del espacio que más contribuyen a la cantidad deseada. Esto resulta especialmente ventajoso cuando la distribución de interés es difícil de muestrear directamente o cuando ciertos eventos raros influyen fuertemente en el resultado.

## Formulación Matemática

1. **Estimador Monte Carlo Estándar**  
   Si se desea calcular $\mathbb{E}_{p}[f(X)]$, donde $X \sim p$, un estimador Monte Carlo convencional es  
   $$
     \hat{\mu}_{\text{MC}} = \frac{1}{N} \sum_{i=1}^{N} f(x_i), \quad x_i \sim p.
   $$

2. **Estimador de Muestreo de Importancia**  
   En lugar de muestrear de $p$, se elige una distribución propuesta $q$ que sea más sencilla de muestrear y no sea nula en regiones donde $p$ lo sea. Entonces,  
   $$
     \mathbb{E}_{p}[f(X)] 
     = \int f(x)\,p(x)\,dx 
     = \int f(x)\,\frac{p(x)}{q(x)}\,q(x)\,dx 
     = \mathbb{E}_{q}\![f(X)\,\omega(X)],
   $$
   donde $\omega(x)=\frac{p(x)}{q(x)}$ es el peso de importancia. Para muestras $x_i\sim q$, el estimador se convierte en  
   $$
     \hat{\mu}_{\text{IS}}
     = \frac{1}{N}\sum_{i=1}^{N} f(x_i)\,\omega(x_i).
   $$

3. **Varianza y Elección de la Propuesta**  
   El muestreo de importancia puede reducir la varianza si $q$ concentra mayor densidad donde $f(x)\,p(x)$ es grande. Sin embargo, si $q$ no se elige adecuadamente, la varianza puede aumentar.

## Ejemplo en Python (1D)

En este ejemplo se considera una distribución objetivo $p(x)$ proporcional a $\exp(-x^2)$ y se elige como propuesta $q(x)$ una distribución normal $\mathcal{N}(0,1)$. Se estima $\mathbb{E}_{p}[f(X)]$ para $f(x)=x^2$.

In [4]:
import numpy as np

def target_pdf(x):
    # p(x) ~ exp(-x^2) sin normalizar
    return np.exp(-x**2)

def proposal_pdf(x, mean=0, std=1):
    # q(x) = N(mean, std^2)
    return (1.0 / np.sqrt(2.0*np.pi*std**2)) * \
           np.exp(-0.5 * ((x - mean)/std)**2)

def f(x):
    return x**2

N = 10000
mean_proposal = 0.0
std_proposal = 1.0

# Muestras de la propuesta q(x)
samples = np.random.normal(mean_proposal, std_proposal, size=N)

# Cálculo de los pesos
weights = target_pdf(samples) / proposal_pdf(samples, mean_proposal, std_proposal)

# Estimador de importancia
estimate_is = np.mean(f(samples) * weights)
print("Estimación con muestreo de importancia:", estimate_is)

# (Opcional) Estimación Monte Carlo "directa" (si fuera posible muestrear de p)
# En realidad, p(x) ~ N(0, 1/2), por lo que se puede simular escalando std.
samples_direct = np.random.normal(0, 1/np.sqrt(2), size=N)
estimate_mc = np.mean(samples_direct**2)
print("Estimación Monte Carlo directa:", estimate_mc)

Estimación con muestreo de importancia: 0.8920752623169319
Estimación Monte Carlo directa: 0.4960962445819775


## Ejemplo en Python (2D)

Aquí se estima $\mathbb{E}[f(X,Y)]$ con $(X,Y)\sim p$ tal que $p(x,y)\propto \exp(-(x^2 + y^2))$. Se elige una propuesta gaussiana 2D.

In [5]:
import numpy as np

def target_pdf_2d(x):
    # x es [x, y]
    return np.exp(-(x[0]**2 + x[1]**2))

def proposal_pdf_2d(x, mean, cov):
    diff = np.array(x) - np.array(mean)
    inv_cov = np.linalg.inv(cov)
    norm_const = 1.0 / (2.0 * np.pi * np.sqrt(np.linalg.det(cov)))
    return norm_const * np.exp(-0.5 * diff.T @ inv_cov @ diff)

def f_2d(x):
    return x[0]**2 + x[1]**2

N = 10000
mean_prop = [0, 0]
cov_prop = [[1, 0],
            [0, 1]]

# Muestras de la propuesta
samples_2d = np.random.multivariate_normal(mean_prop, cov_prop, size=N)

# Pesos
weights_2d = np.array([
    target_pdf_2d(s) / proposal_pdf_2d(s, mean_prop, cov_prop) 
    for s in samples_2d
])

# Estimación
estimate_is_2d = np.mean([f_2d(s)*w for s,w in zip(samples_2d, weights_2d)])
print("Estimación 2D con muestreo de importancia:", estimate_is_2d)

Estimación 2D con muestreo de importancia: 3.120392584335414


## Ejercicios

1. **Integral 1D con Distintas Propuestas**  
   - Definir $p(x)\propto \exp(-x^2)$.  
   - Escoger dos propuestas distintas, por ejemplo, $\mathcal{N}(0,1)$ y $\mathcal{N}(2,1)$.  
   - Implementar muestreo de importancia con ambas y comparar el valor estimado y su varianza.  
   - Discutir cuál propuesta resulta más eficaz y por qué.

2. **Ejemplo 2D y Visualización de Pesos**  
   - Sea $p(x,y)\propto \exp(-(x^2 + y^2))$.  
   - Usar como propuesta una distribución 2D desviada respecto al origen.  
   - Calcular y graficar los pesos $\omega_i$.  
   - Analizar la presencia de valores extremos en los pesos y su efecto en la varianza.

3. **Probabilidad de Evento Raro**  
   - Calcular la probabilidad $P(X>3)$ donde $X\sim \mathcal{N}(0,1)$ usando Monte Carlo estándar.  
   - Emplear muestreo de importancia usando $\mathcal{N}(3,1)$ como propuesta.  
   - Comparar resultados y varianzas.  
   - Experimentar con $\mathcal{N}(10,1)$ y observar el efecto de una propuesta alejada.

4. **Escenario con Cobertura Parcial**  
   - Definir $p(x)\propto \exp(-x)$ en $(0,\infty)$.  
   - Escoger $q(x)$ mal diseñada que no cubra toda la región positiva.  
   - Implementar muestreo de importancia y observar el sesgo.  
   - Corregir la propuesta para cubrir $(0,\infty)$ y comparar resultados.

# 3. Muestreo Estratificado

Los métodos de Monte Carlo estiman integrales o valores esperados muestreando aleatoriamente una función sobre un dominio. Aunque el muestreo aleatorio simple converge con una tasa proporcional a $1 / \sqrt{n}$, puede presentar bastante varianza en etapas tempranas o para funciones con regiones de alta variabilidad. El **muestreo estratificado** busca reducir dicha varianza dividiendo el dominio de integración en subregiones (estratos) y muestreando cada subregión de forma más controlada.

Puntos clave:

- En un muestreo Monte Carlo simple, las muestras pueden reunirse de manera desigual, ignorando la variabilidad en ciertas partes del dominio.  
- El muestreo estratificado divide el dominio en subregiones (estratos), asegurando muestras en cada estrato para cubrir el dominio de forma más sistemática.  
- A menudo, la varianza disminuye en comparación con el muestreo completamente aleatorio.

## Formulación Matemática

Sea el objetivo estimar
$$
I = \int_{\Omega} f(\mathbf{x}) \, d\mathbf{x},
$$
donde $\Omega$ es el dominio de integración. El muestreo estratificado particiona $\Omega$ en $k$ estratos disjuntos $\Omega_1, \Omega_2, \dots, \Omega_k$ tales que
$$
\bigcup_{j=1}^k \Omega_j = \Omega, \quad 
\Omega_j \cap \Omega_{j'} = \varnothing \quad (j \neq j').
$$
Sea $p_j = \text{Vol}(\Omega_j)/\text{Vol}(\Omega)$ la fracción de “volumen” total que aporta el estrato $\Omega_j$. Si se toman $n_j$ muestras en cada estrato, una elección común es la asignación proporcional: $n_j = n \cdot p_j$.

Dentro de cada estrato $\Omega_j$, se muestrean puntos $\mathbf{x}_{j,i}$ ($i = 1, \ldots, n_j$) y se estima
$$
I_j = \frac{1}{n_j} \sum_{i=1}^{n_j} \frac{f(\mathbf{x}_{j,i})}{q_j(\mathbf{x}_{j,i})},
$$
donde $q_j(\mathbf{x})$ es la función de densidad de muestreo restringida al estrato $\Omega_j$. El estimador estratificado global es
$$
\hat{I} = \sum_{j=1}^k p_j \, I_j.
$$
Bajo condiciones suaves, este estimador tiene una varianza menor que el muestreo Monte Carlo simple.

### Ejemplo Unidimensional

**Objetivo**: Estimar
$$
I = \int_0^1 g(x)\, dx
$$
usando muestreo estratificado.

1. **Definir la función integrando** (por ejemplo, $g(x) = \sin(5x)$ u otra).  
2. **Particionar** $[0,1]$ en $k$ estratos de igual tamaño ($\frac{1}{k}$).  
3. **Tomar** $\frac{n}{k}$ muestras en cada subintervalo.

In [6]:
import numpy as np

def g_1d(x):
    return np.sin(5 * x)  # Función de ejemplo

def muestreo_estratificado_1d(num_muestras=1000, k=10):
    # k estratos, cada uno de tamaño 1/k
    muestras_por_estrato = num_muestras // k
    estimaciones = []

    for j in range(k):
        x_inferior = j / k
        x_superior = (j + 1) / k

        # Tomar muestras en el subintervalo
        xs = np.random.uniform(x_inferior, x_superior, muestras_por_estrato)
        valores = g_1d(xs)

        # Promedio local en este estrato
        promedio_local = np.mean(valores)

        # Ponderado por el tamaño del estrato
        estimaciones.append(promedio_local / k)

    return np.sum(estimaciones)

# Ejecución
n = 10000
estimacion_1d = muestreo_estratificado_1d(num_muestras=n, k=10)
print("Estimación con muestreo estratificado (1D):", estimacion_1d)

Estimación con muestreo estratificado (1D): 0.14389503324986236


### Ejemplo Bidimensional

**Objetivo**: Estimar
$$
I = \iint_{[0,1]^2} f(x,y)\, dx\, dy
$$
con muestreo estratificado. Por ejemplo,
$$
f(x, y) = \exp\bigl(-(x^2 + y^2)\bigr).
$$

1. **Particionar** el cuadrado unitario $[0,1]^2$ en una malla de $k \times k$ subcuadrados.  
2. **Muestrear** en cada subcuadrado.


In [7]:
import numpy as np

def f_2d(x, y):
    return np.exp(-(x**2 + y**2))

def muestreo_estratificado_2d(num_muestras=10000, k=10):
    # k x k subcuadrados
    muestras_por_celda = num_muestras // (k * k)
    estimacion_total = 0.0

    for i in range(k):
        for j in range(k):
            x_inferior, x_superior = i / k, (i + 1) / k
            y_inferior, y_superior = j / k, (j + 1) / k

            # Muestra en la subcelda
            xs = np.random.uniform(x_inferior, x_superior, muestras_por_celda)
            ys = np.random.uniform(y_inferior, y_superior, muestras_por_celda)

            valores = f_2d(xs, ys)
            promedio_local = np.mean(valores)

            # Cada subcuadrado tiene área 1/(k^2)
            area_subcuadrado = 1.0 / (k**2)
            estimacion_total += promedio_local * area_subcuadrado

    return estimacion_total

estimacion_2d = muestreo_estratificado_2d(num_muestras=10000, k=10)
print("Estimación con muestreo estratificado (2D):", estimacion_2d)

Estimación con muestreo estratificado (2D): 0.5578892505005325


## Ejercicios Propuestos

1. **Muestreo Estratificado vs. Muestreo Crudo (1D)**  
   - Implementa tanto el muestreo crudo como el estratificado para estimar la integral de una función a tu elección en [0,1].  
   - Repite cada enfoque varias veces (por ejemplo, 50 a 100 repeticiones) y registra la desviación estándar de las estimaciones.  
   - Compara las medias y desviaciones estándar de ambos métodos y explica por qué el muestreo estratificado puede reducir la varianza.

2. **Optimización del Número de Estratos**  
   - En 1D, varía el número de estratos k (por ejemplo, 2, 5, 10, 20) para un número fijo de muestras totales.  
   - Para cada k, estima la integral (puedes usar más de una función) y mide el error promedio y la varianza a lo largo de varias repeticiones.  
   - Haz un gráfico o tabla de la varianza en función de k, y comenta cómo el incremento de k influye en los resultados.  
   - Identifica un punto en el que incrementar k deja de brindar beneficios significativos y argumenta posibles razones.

3. **Integración Estratificada en 2D**  
   - Escribe código para integrar una función 2D en [0,1]×[0,1] usando muestreo estratificado.  
   - Compara el resultado con (1) muestreo crudo y (2) una solución analítica (si la función lo permite).  
   - Sub-ejercicio: Intenta utilizar más estratos en el eje donde la función muestre mayor variabilidad. Describe el efecto sobre la reducción de la varianza.

4. **Practicidad en Dimensiones Elevadas**  
   - Extiende tu código a 3D o 4D, usando el mismo número total de muestras.  
   - Observa cuán rápido crece el número de estratos y cómo esto impacta los tiempos de ejecución.  
   - Sub-ejercicio: Introduce un enfoque adaptativo o jerárquico (por ejemplo, subdividir más en estratos de alta varianza). Compara la precisión y el tiempo de ejecución con el estratificado uniforme, y describe las compensaciones observadas.

# Variantes de Control

Las **variantes de control** (control variates) son una técnica potente para reducir la varianza en simulaciones de Monte Carlo. La idea principal consiste en introducir una variable aleatoria auxiliar (el “control”) cuya esperanza es conocida y que está correlacionada con la cantidad de interés. Al restar una forma ponderada de ese control de nuestro estimador original, se puede obtener una estimación de menor varianza sin cambiar el valor esperado.

**Puntos clave a introducir:**

- Cómo los estimadores de Monte Carlo (MC) pueden presentar varianzas altas, especialmente en sistemas complejos.  
- El concepto central de variantes de control: sumar y restar una función de media conocida que además esté correlacionada con la función objetivo.  
- Cómo la correlación entre la variante de control y la función objetivo determina la reducción de varianza.  
- La fórmula para el peso óptimo.

## Formulación Matemática

Sea $$X$$ una variable aleatoria con distribución $$p(x)$$. Queremos estimar
$$
\theta \;=\; \mathbb{E}[V(X)] \;=\; \int V(x)\,p(x)\,dx.
$$
El estimador de Monte Carlo estándar es
$$
\hat{\theta} \;=\; \frac{1}{N}\sum_{k=1}^N V(X_k),
$$
donde $X_1, X_2, \dots, X_N$ son muestras independientes de $p(x)$.

Una variante de control es una función $W(X)$ con esperanza conocida $\mathbb{E}[W(X)] = \mu$. Definimos:
$$
U(X) \;=\; V(X) \;-\; \alpha \bigl(W(X) - \mu \bigr),
$$
donde $\alpha$ es una constante que debemos elegir. El nuevo estimador de Monte Carlo se vuelve:
$$
\hat{\theta}_{\text{cv}} \;=\; \frac{1}{N}\sum_{k=1}^N \Bigl(V(X_k) - \alpha\bigl(W(X_k) - \mu\bigr)\Bigr) + \alpha\,\mu.
$$
La esperanza de este estimador sigue siendo $\theta$ para cualquier $\alpha$. Su varianza se minimiza cuando
$$
\alpha^* \;=\; \frac{\mathrm{Cov}(V(X),\,W(X))}{\mathrm{Var}(W(X))}.
$$
En la práctica, $\alpha^*$ se desconoce y debe estimarse usando las mismas muestras de Monte Carlo o corridas de prueba.

## Ejemplos en Python paso a paso

A continuación se muestran fragmentos de código ilustrativos. Se enfoca en la generación de muestras, la aplicación de la variante de control y la estimación de $\alpha$. El nivel de detalle puede ajustarse según la audiencia.

## Ejemplo en 1D

In [8]:
import numpy as np

def funcion_objetivo(x):
    # Supongamos que queremos la esperanza de x^2 + np.sin(x)
    return x**2 + np.sin(x)

def funcion_control(x):
    # Usamos x^2 como control, cuya media en [0,1] se conoce
    return x**2

N = 10_000
X = np.random.rand(N)  # Muestras uniformes en [0,1]

V_muestras = funcion_objetivo(X)
W_muestras = funcion_control(X)

# Media verdadera de x^2 en [0,1] = 1/3
media_control = 1/3

cov_VW = np.cov(V_muestras, W_muestras, bias=True)[0,1]
var_W = np.var(W_muestras, ddof=0)

alpha = cov_VW / var_W

estimacion_cv = np.mean(V_muestras - alpha*(W_muestras - media_control)) + alpha*media_control
estimacion_mc = np.mean(V_muestras)

print("Estimación MC estándar:", estimacion_mc)
print("Estimación con variante de control:", estimacion_cv)

Estimación MC estándar: 0.789904607451731
Estimación con variante de control: 1.3890139361473501


## Ejemplo en 2D

In [9]:
import numpy as np

def funcion_objetivo_2d(x, y):
    # Ejemplo: x*y + np.sin(x+y)
    return x*y + np.sin(x+y)

def funcion_control_2d(x, y):
    # Usamos x*y como control. Si X e Y son N(0,1) independientes, la media es 0
    return x*y

N = 10_000
X = np.random.randn(N)
Y = np.random.randn(N)

V_muestras_2d = funcion_objetivo_2d(X, Y)
W_muestras_2d = funcion_control_2d(X, Y)

media_control_2d = 0.0

cov_VW_2d = np.cov(V_muestras_2d, W_muestras_2d, bias=True)[0,1]
var_W_2d = np.var(W_muestras_2d, ddof=0)

alpha_2d = cov_VW_2d / var_W_2d if var_W_2d != 0 else 0

estimacion_cv_2d = np.mean(V_muestras_2d - alpha_2d*(W_muestras_2d - media_control_2d)) + alpha_2d*media_control_2d
estimacion_mc_2d = np.mean(V_muestras_2d)

print("Estimación MC estándar (2D):", estimacion_mc_2d)
print("Estimación con variante de control (2D):", estimacion_cv_2d)

Estimación MC estándar (2D): -0.006121597779753133
Estimación con variante de control (2D): -0.008071857483568009


## Ejemplo en ND

In [10]:
import numpy as np

def funcion_objetivo_nd(muestras):
    # 'muestras' tendrá forma (N, d)
    # Ejemplo: suma de sin en cada dimensión
    return np.sum(np.sin(muestras), axis=1)

def funcion_control_nd(muestras):
    # Usamos la suma de los valores como control
    return np.sum(muestras, axis=1)

def variantes_control_nd(d, N=10_000):
    # Creamos muestras de dimensión d de una normal estándar
    X = np.random.randn(N, d)

    V_muestras = funcion_objetivo_nd(X)
    W_muestras = funcion_control_nd(X)

    # La media de la suma de d normales estándar es 0
    media_control = 0.0

    cov_VW = np.cov(V_muestras, W_muestras, bias=True)[0,1]
    var_W = np.var(W_muestras, ddof=0)
    alpha = cov_VW / var_W if var_W != 0 else 0

    estimacion_cv = np.mean(V_muestras - alpha*(W_muestras - media_control)) + alpha*media_control
    estimacion_mc = np.mean(V_muestras)
    
    return estimacion_mc, estimacion_cv

if __name__ == "__main__":
    d = 5
    mc_est, cv_est = variantes_control_nd(d, N=20_000)
    print(f"Estimación MC estándar (d={d}):", mc_est)
    print(f"Estimación con variante de control (d={d}):", cv_est)

Estimación MC estándar (d=5): 0.00015350407578632836
Estimación con variante de control (d=5): -0.0028752864726057665


## Ejercicios Propuestos

**Ejercicio 1**  

1. Implementa un estimador de Monte Carlo para aproximar la integral  
   $$\int_{0}^{1} x^2 \, dx.$$  
   Compara con la solución exacta.  
2. Preguntas:  
   - ¿Cómo cambia la varianza de la estimación con diferentes tamaños de muestra?  
   - ¿Qué factores influyen en la precisión del resultado?

**Ejercicio 2**  

1. Aplica una variante de control al mismo estimador del Ejercicio 1.  
   - Usa $W(x) = x$ como función de control (media conocida: 0.5 en [0,1]).  
2. Preguntas:  
   - Estima $\alpha$ a partir de las muestras. ¿Cuánto mejora el resultado en comparación con el método estándar?  
   - ¿Qué pasa con la reducción de varianza si cambias $W(x)$ por una función poco correlacionada?

**Ejercicio 3**  

1. Pasa a un escenario 2D donde $(X, Y)$ siguen una distribución normal bivariante con correlación $\rho$.  
   - Define una función $V(X, Y)$.  
   - Elige una función de control $W(X, Y)$ con media conocida.  
2. Preguntas:  
   - ¿Cómo se calcula $\mathrm{Cov}(V, W)$ en la práctica?  
   - ¿Cómo afecta el valor de $\rho$ a la efectividad de la variante de control?  
   - ¿Cómo se compara la reducción de varianza para diferentes valores de $\rho$?

**Ejercicio 4**  

1. Aplica variantes de control en un caso de mayor dimensión (3D o más).  
   - Por ejemplo, genera muestras de $\mathcal{N}(0, I)$ en dimensión $d$.  
   - Define $V(\mathbf{X}) = \sum \sin(X_i)$ o una función similar.  
2. Preguntas:  
   - Propón una variante de control y explica por qué podría estar correlacionada con $V$.  
   - Compara el desempeño de tu método contra el Monte Carlo estándar para distintas dimensiones.  
   - ¿Cuáles son las dificultades al estimar $\alpha$ en alta dimensión y cómo podrías mitigarlas

# Variantes Antitéticas

Los métodos de Monte Carlo (MC) aproximan integrales o expectativas muestreando al azar desde una distribución de probabilidad y tomando promedios empíricos. La técnica de *variantes antitéticas* (Antithetic Variates) es un método de **reducción de varianza**. En lugar de muestrear puntos completamente al azar, se generan pares de entradas aleatorias correlacionadas negativamente. Esta correlación reduce la fluctuación estadística del estimador, lo que puede dar resultados más precisos con el mismo número de muestras.

Cuando se usan variantes antitéticas:

1. Cada punto aleatorio $U$ se empareja con un punto transformado $\bar{U}$ de modo que ambos estén correlacionados negativamente.  
2. Se calcula el promedio de la función en ambos puntos, reduciendo así la varianza global del estimador.

A continuación, se muestra una propuesta de plan de lección que incluye ejemplos en Python para sistemas de 1D, 2D y ND, así como análisis de las dificultades potenciales.

## Formulación Matemática

Dada una muestra aleatoria $\mathbf{Z}$, se construye su pareja antitética $\mathbf{Z}^*$ (a menudo $\mathbf{Z}^* = -\mathbf{Z}$ en distribuciones simétricas). Si se desea estimar $\mu = \mathbb{E}[f(\mathbf{Z})]$, el estimador antitético es:
$$
\hat{\mu} = \frac{1}{N}\sum_{i=1}^{N} \frac{f(\mathbf{Z}_i) + f(\mathbf{Z}_i^*)}{2}.
$$
Si $\operatorname{Cov}(f(\mathbf{Z}), f(\mathbf{Z}^*)) < 0$, la varianza del promedio $\tfrac{1}{2}\bigl(f(\mathbf{Z}) + f(\mathbf{Z}^*)\bigr)$ es menor que la varianza de usar un solo punto.

### Ejemplo Unidimensional en Python**

Supongamos que queremos estimar la integral:
$$
I = \int_0^1 x^3 \, dx.
$$
El valor exacto es $I = 0.25$. Implementamos un estimador Monte Carlo con y sin variantes antitéticas:

In [11]:
import numpy as np

def f(x):
    return x**3

N = 10_000

# Monte Carlo estándar
u = np.random.rand(N)
estimate_mc = np.mean(f(u))

# Variantes antitéticas
# Emparejamos u_i con 1 - u_i
u1 = np.random.rand(N//2)
u2 = 1.0 - u1
estimate_av = np.mean( (f(u1) + f(u2))/2.0 )

print("Valor exacto:              ", 0.25)
print("Estimación MC estándar:    ", estimate_mc)
print("Estimación antitética:     ", estimate_av)

Valor exacto:               0.25
Estimación MC estándar:     0.2474158062722032
Estimación antitética:      0.2492623694418049


1. Observar si la estimación antitética fluctúa menos que la estimación Monte Carlo estándar.  
2. Explicar cómo surge la correlación negativa, ya que $x + (1-x)=1$ produce correlación negativa para muchas funciones monótonas.

### Ejemplo Bidimensional**

Consideremos:
$$
I = \iint_{[0,1]^2} (x + y)^3 \, dx\,dy.
$$
Podemos calcular el valor exacto analíticamente, pero usaremos Monte Carlo para ilustrar. Sea $(U_1, U_2)$ uniforme en $[0,1]^2$. Su pareja antitética es $(1-U_1, 1-U_2)$.

In [12]:
import numpy as np

def f_2d(x, y):
    return (x + y)**3

N = 10_000
u = np.random.rand(N, 2)  # shape (N,2)

# MC estándar
vals_mc = [f_2d(*pt) for pt in u]
estimate_mc_2d = np.mean(vals_mc)

# Antitético
u_half = np.random.rand(N//2, 2)
u_antithetic = 1.0 - u_half
vals_av = [(f_2d(*pt1) + f_2d(*pt2))/2.0 
           for pt1, pt2 in zip(u_half, u_antithetic)]
estimate_av_2d = np.mean(vals_av)

print("Aprox (MC estándar):  ", estimate_mc_2d)
print("Aprox (antitético):   ", estimate_av_2d)

Aprox (MC estándar):   1.4672723383513167
Aprox (antitético):    1.5057870918793264


### Ejemplo ND

En dimensiones más altas (por ej., $d$ dimensiones uniforme en $[0,1]^d$), la construcción es similar. Si el vector aleatorio es $\mathbf{U} = (U_1, \dots, U_d)$, la versión antitética es $(1-U_1, \dots, 1-U_d)$.

Aquí hay un ejemplo en Python para la función $f(\mathbf{x}) = \|\mathbf{x}\|^2$ en dimensión $d$:

In [13]:
import numpy as np

def f_nd(x):
    # x es un array 1D con dimensión d
    return np.sum(x**2)

d = 5     # dimensión
N = 20_000

U = np.random.rand(N, d)
vals_mc_nd = [f_nd(u_i) for u_i in U]
estimate_mc_nd = np.mean(vals_mc_nd)

U_half = np.random.rand(N//2, d)
U_antithetic = 1.0 - U_half
vals_av_nd = [(f_nd(uh) + f_nd(ua))/2.0 
              for uh, ua in zip(U_half, U_antithetic)]
estimate_av_nd = np.mean(vals_av_nd)

print("Dimensión:", d)
print("Estimación MC estándar:", estimate_mc_nd)
print("Estimación antitética: ", estimate_av_nd)

Dimensión: 5
Estimación MC estándar: 1.6673688503622477
Estimación antitética:  1.6681010924764887


## **Ejercicios Propuestos**

### **Ejercicio 1: Integral 1D con Comparación Pareada**

1. Implementar un estimador Monte Carlo para aproximar  
   $$
   I = \int_{0}^{1} \frac{1}{1 + x} \, dx 
   $$
   usando tanto el método estándar como el antitético, donde $u' = 1 - u$.  

2. Comparar sus estimaciones y varianzas empíricas en varias corridas.  

3. Analizar si al aumentar el número de muestras se reduce de forma consistente la brecha de varianza.  

4. Reflexionar sobre posibles situaciones donde las variantes antitéticas no reduzcan la varianza.

### **Ejercicio 2: Verificación de Covarianza Negativa**

1. Escoger una función $f(x)$ estrictamente creciente (por ejemplo, $f(x) = x^3$) y generar pares $(u, 1-u)$.  
2. Calcular la covarianza muestral entre $\{f(u_i)\}$ y $\{f(1-u_i)\}$.  
3. Verificar si la covarianza resulta negativa.  
4. Explicar brevemente la relevancia de la covarianza en el rendimiento de las variantes antitéticas.

### **Ejercicio 3: Estimación en 2D**

1. Definir la integral 2D:  
   $$
   I = \iint_{[0,1]^2} (x + y)^2 \, dx \, dy.
   $$

2. Implementar un estimador Monte Carlo estándar.  

3. Implementar la versión antitética emparejando $(u_1, u_2)$ con $(1-u_1, 1-u_2)$.  

4. Comparar las varianzas numéricas para ver si las variantes antitéticas ayudan.  

5. Explicar por qué la magnitud de la reducción de varianza varía con diferentes funciones incluso en la misma dimensión.

### **Ejercicio 4: Experimento en Altas Dimensiones y Dificultades**

1. Elegir una dimensión $d \ge 5$ y la función  
   $$
   f(\mathbf{x}) = e^{-\|\mathbf{x}\|^2}, \quad \mathbf{x} \in [0,1]^d.
   $$

2. Crear un estimador Monte Carlo con $N$ puntos $\mathbf{x}_i$.  

3. Construir la muestra antitética $\mathbf{x}_i^* = \mathbf{1} - \mathbf{x}_i$ y calcular el estimador antitético.  

4. Examinar si la correlación negativa entre $f(\mathbf{x}_i)$ y $f(\mathbf{x}_i^*)$ reduce la varianza de forma apreciable.  

5. Discutir cómo la dimensionalidad alta puede disminuir los beneficios del muestreo antitético.
